# MTpy Example 01

## Transfer Function Collection

**Goal:** demonstrate how to build a standardized collection of tranfer functions from various file types ("EDI", "EMTFXML").

**Test Data:** Yellowstone, WY, USA 

   - Broadband data provided by University of Washington and Oregon State University processed by Paul Bedrosian (USGS)
   - Earthscope long period data archived at IRIS
   - Long period data from DeGroot-Hedlin archived at IRIS
    

We will build an `MTCollection` using `MTpy v2.0`.  Under the hood `MTCollection` is storing the transfer functions in an MTH5 file and using `mt_metadata` to read the transfer functions into a standardize transfer function.  We will also have a look at how to adjust some metadata so that your transfer functions can be nicely organized.  Each of these data sets were collected as different surveys and we will store them as such.  We will go survey by survey loading in the transfer functions.



In [1]:
from copy import deepcopy
from pathlib import Path

from mtpy import MT, MTCollection

## Creat Transfer Function Collection

The first thing to do is to open a new `MTCollection` object.  You can call it what you want, for now we will call it `yellowstone_mt_collection`.

### Add Earthscope Data

The data is provided locally, but the data are publicly available on the [IRIS EMTF SPUD](http://www.ds.iris.edu/spud/). Just draw a box around the area of interest and click `download spud bundle`.  This will give you a zip file that contains an EDI, EMTFXML, ZRR file and an image of what the response should look like.  We have copied the EMTFXML files into a single folder for convenience.  

### Broadband Data

First we will load in broadband transfer functions which are in EDI format. These files have no `survey.id` type information, therefore we are going to add it.  These files are stored locally on your machine, so just adjust the file path.  We will loop over each file and add in the appropriate metadata 

In [2]:
%%time
with MTCollection() as mc:
    mc.open_collection(
        filename=Path().cwd().parent.parent.joinpath("data", "transfer_functions", "yellowstone_mt_collection_02.h5")
    )
    # broadband data
    bb_edi_path = Path().cwd().parent.parent.joinpath("data", "transfer_functions","broadband")
    for edi_filename in mc.make_file_list(bb_edi_path, file_types=["edi"]):
        mt_object = MT(edi_filename)
        mt_object.read()

        # update the survey id
        mt_object.survey_metadata.id = "YSBB"
        mc.add_tf(mt_object)

    ## long period data from the MTArray
    earthscope_path = Path().cwd().parent.parent.joinpath("data", "transfer_functions","earthscope")
    for xml_fn in mc.make_file_list(earthscope_path, file_types=["xml"]):
        mt_object = MT(xml_fn)
        mt_object.read()
        mc.add_tf(mt_object)

        tf_df = mc.dataframe

24:09:26T10:28:10 | INFO | line:679 |mth5.mth5 | _initialize_file | Initialized MTH5 0.2.0 file c:\Users\jpeacock\OneDrive - DOI\Documents\GitHub\iris-mt-course-2022\data\transfer_functions\yellowstone_mt_collection_02.h5 in mode a
24:09:26T10:28:10 | WARNING | line:1080 |mth5.mth5 | get_survey | /Experiment/Surveys/YSBB does not exist, check survey_list for existing names.
24:09:26T10:30:40 | WARNING | line:84 |mt_metadata.transfer_functions.io.emtfxml.metadata.helpers | _read_element | No declination in EMTF XML
24:09:26T10:30:41 | WARNING | line:1080 |mth5.mth5 | get_survey | /Experiment/Surveys/Transportable_Array does not exist, check survey_list for existing names.
24:09:26T10:30:41 | WARNING | line:206 |mt_metadata.transfer_functions.tf.station | get_run | Could not find IDD11a in runs.
24:09:26T10:30:41 | WARNING | line:206 |mt_metadata.transfer_functions.tf.station | get_run | Could not find IDD11c in runs.
24:09:26T10:30:42 | WARNING | line:84 |mt_metadata.transfer_functions.

### Make sure everything loaded in properly

We want to make sure everything loaded in properly.  `MTCollection` has a convenient property called `dataframe` this will list all transfer functions found in the MTH5 file.  

**Note:** This is a property that when called returns a summary of all transfer functions contained in the MTH5 at the point that it is called.  Therefore, if you update the file by adding a transfer function and then calle `dataframe` you will get an updated summary.  There is more we can do with `dataframe` which we will look at in a later lesson.

In [3]:
tf_df

,station,survey,latitude,longitude,elevation,tf_id,units,has_impedance,has_tipper,has_covariance,period_min,period_max,hdf5_reference,station_hdf5_reference
0,IDD11,Transportable_Array,47.043986,-116.345982,850.750,IDD11,none,True,True,True,7.31429,18724.57,<HDF5 object reference>,<HDF5 object reference>
1,IDD12,Transportable_Array,47.047800,-115.348000,1137.862,IDD12,none,True,True,True,7.31429,18724.57,<HDF5 object reference>,<HDF5 object reference>
2,IDE11,Transportable_Array,46.353000,-116.212965,882.625,IDE11,none,True,True,True,7.31429,18724.57,<HDF5 object reference>,<HDF5 object reference>
3,IDE12,Transportable_Array,46.387100,-115.582000,1633.900,IDE12,none,True,True,True,7.31429,18724.57,<HDF5 object reference>,<HDF5 object reference>
4,IDF11,Transportable_Array,45.889496,-116.157113,1175.550,IDF11,none,True,True,True,7.31429,18724.57,<HDF5 object reference>,<HDF5 object reference>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,SR930,Yellowstone-Snake_River_Plain,43.188800,-113.042300,1568.050,SR930,none,True,True,True,7.31429,18724.57,<HDF5 object reference>,<HDF5 object reference>
217,SR954,Yellowstone-Snake_River_Plain,43.689700,-112.375700,1536.925,SR954,none,True,True,True,7.31429,18724.57,<HDF5 object reference>,<HDF5 object reference>
218,SR966,Yellowstone-Snake_River_Plain,43.948300,-112.039000,1486.875,SR966,none,True,True,True,7.31429,18724.57,<HDF5 object reference>,<HDF5 object reference>
219,SR980,Yellowstone-Snake_River_Plain,44.258100,-111.553000,2075.900,SR980,none,True,True,True,7.31429,18724.57,<HDF5 object reference>,<HDF5 object reference>


In [4]:
tf_df.survey.unique()

array(['Transportable_Array', 'YSBB', 'Yellowstone-Snake_River_Plain'],
      dtype=object)

## Next Example

In the next we will have a look at the various plotting functions `MTCollection` has and how to use them. 